# Data Pre-Processing

In [1]:
import spacy
import pandas as pd
from tqdm.notebook import tqdm

In [70]:
df = pd.read_pickle('data_merged.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 971264 entries, 0 to 971279
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   text            971264 non-null  object        
 1   date            971264 non-null  datetime64[ns]
 2   speaker_name    971264 non-null  object        
 3   speaker_gender  971264 non-null  object        
 4   speaker_role    884850 non-null  object        
 5   speaker_party   969563 non-null  object        
 6   comments        971264 non-null  object        
 7   id              971264 non-null  object        
 8   speaker_id      971264 non-null  int64         
 9   speech_length   971264 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 81.5+ MB


In [ ]:
#!python -m spacy download de_core_news_sm # downloaded successfully

In [17]:
nlp = spacy.load('de_core_news_sm') 
doc = nlp(df.text[0])
doc

  Sehr geehrter Herr Alterspräsident! So muss ich es jetzt sagen, genau; ich muss mich daran gewöhnen. Nicht lange.  Nicht lange; da haben Sie schon recht. – Liebe Kolleginnen und Kollegen! Meine sehr geehrten Damen und Herren! Die Übernahme der Geschäftsordnung ist eine der ersten wichtigen Entscheidungen, die wir heute zu Beginn der Wahlperiode treffen müssen. Die Geschäftsordnung ist Grundlage für unsere gemeinsame Arbeit. Sie hat sich über viele Wahlperioden bewährt und wird uns auch durch die neue Wahlperiode tragen. Es ist gute Tradition, dass wir die Geschäftsordnung am Anfang einer Wahlperiode mit breiter parlamentarischer Mehrheit übernehmen, und das sollten wir auch heute tun. Sie ist nicht in Stein gemeißelt – das wissen diejenigen, die bereits in der letzten Wahlperiode hier im Bundestag tätig waren –, sondern die Geschäftsordnung ist immer ein, ich sage mal, lebendiges Dokument, das sich den Gegebenheiten des Parlamentarismus, aber auch den Gegebenheiten der Arbeit hier im

In [21]:
def keep_token(token):
    include_pos = ['NOUN', 'PROPN']
    exclude_tokens = ['kollege', 'kollegin', 'abgeordneter', 'redner', 'rednerin', 'staatssekretär', 'staatssekretärin', 'minister', 'ministerin', 'bundesminister', 'dame', 'herr']

    if token.is_alpha and token.ent_type_ != 'PER' and token.pos_ in include_pos and not (token.is_stop or token.lemma_.lower() in exclude_tokens):
       return True
    return False

'alterspräsident übernahme geschäftsordnung entscheidung beginn wahlperiode geschäftsordnung grundlage arbeit wahlperiode wahlperiode tradition geschäftsordnung anfang wahlperiode mehrheit stein wahlperiode bundestag geschäftsordnung dokument gegebenheit parlamentarismus gegebenheit arbeit bundestag geschäftsordnung entwicklung wahlperiode änderung reform verhaltensregeln initiative präsident geschäftsordnung wahlperiode generalüberholung impuls fraktion weg vorschlag opposition geschäftsordnung legislaturperiode regelung ausschuß antrag absenken quor beschlussfähigkeit bundestag geschäftsordnung zustimmung antrag weitergeltung geschäftsordnung änderung nächster wort'

In [22]:
speeches_processed = []

for speech in tqdm(nlp.pipe(df.text, n_process=6, batch_size=64)):
    tokens = [token.lemma_.lower() for token in speech if keep_token(token)]
    speeches_processed.append(tokens)

0it [00:00, ?it/s]

In [23]:
len(speeches_processed)

971264

## Save & load processed speeches

In [4]:
import pickle

# save processed speeches to a file
with open('speeches_processed.pkl', 'wb') as f:
   pickle.dump(speeches_processed, f)

# load speeches_processed
with open('speeches_processed.pkl', 'rb') as f:
   loaded_speeches = pickle.load(f)

In [4]:
len(loaded_speeches)

971264

# Topic Modelling

In [ ]:
#!pip install gensim

In [5]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

In [6]:
dictionary = Dictionary(loaded_speeches)

In [7]:
dictionary.num_docs

971264

In [8]:
dictionary.filter_extremes(no_below=30, no_above=0.2)

In [9]:
len(dictionary)

49455

In [10]:
corpus = [dictionary.doc2bow(doc) for doc in loaded_speeches]

In [11]:
num_topics = 150

In [12]:
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50,
                         num_topics=num_topics, workers=8, passes=15, random_state=20)

In [27]:
lda_model.print_topics()

[(0,
  '0.098*"zusatzfrage" + 0.072*"antwort" + 0.032*"beantwortung" + 0.031*"bundesministerium" + 0.031*"geschäftsbereich" + 0.026*"anlage" + 0.024*"danke" + 0.020*"anfrage" + 0.019*"sitzung" + 0.019*"fragestunde"'),
 (1,
  '0.041*"milliarde" + 0.037*"land" + 0.030*"haushalt" + 0.028*"million" + 0.023*"dm" + 0.019*"bund" + 0.010*"investition" + 0.010*"geld" + 0.010*"bereich" + 0.009*"entwicklung"'),
 (2,
  '0.015*"debatte" + 0.015*"cdu" + 0.013*"haus" + 0.013*"regierung" + 0.012*"parlament" + 0.012*"partei" + 0.011*"bundestag" + 0.011*"rede" + 0.010*"spd" + 0.010*"präsident"'),
 (3,
  '0.059*"kind" + 0.058*"frau" + 0.029*"familie" + 0.016*"bildung" + 0.016*"mensch" + 0.014*"eltern" + 0.013*"schule" + 0.013*"gesellschaft" + 0.013*"jugendliche" + 0.010*"mann"'),
 (4,
  '0.063*"fraktion" + 0.048*"ausschuss" + 0.040*"beratung" + 0.038*"drucksache" + 0.029*"antrag" + 0.028*"beschlussempfehlung" + 0.026*"bündnis" + 0.025*"ausschuß" + 0.021*"grüne" + 0.021*"linke"'),
 (5,
  '0.020*"energie" 

# Coherence Model

## CV Coherence Score

In [ ]:
# between 0 and 1
coherence_model_lda = CoherenceModel(model=lda_model, texts=loaded_speeches, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

## UMass Coherence Score

In [ ]:
# between -14 and 14
corpus_lda = lda_model[corpus]

cm = CoherenceModel(model=lda_model, corpus=corpus_lda, dictionary=dictionary, coherence='u_mass')
cm.get_coherence()

# Visualization with pyLDAvis

In [ ]:
#!pip install pyldavis
import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.gensim

In [ ]:
# create topic distance visualization 
pyLDAvis.enable_notebook()
topic_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
topic_vis

In [ ]:
# save visualization to html file
pyLDAvis.save_html(topic_vis, f"pyLDAvis/lda_{num_topics}_topics.html")

# Save Topic Model

In [ ]:
# save LDA model
with open(f'models/lda_model_{num_topics}.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

# Add Topic Distribution to Dataframe

In [35]:
# load LDA model
with open(f'models/lda_model_150.pkl', 'rb') as f:
    loaded_lda_model = pickle.load(f)

In [ ]:
# print topics for saving to text file (topics.txt)
for i in range(0, loaded_lda_model.num_topics):
    print(f"Topic {i}: " + loaded_lda_model.print_topic(i))

In [56]:
# get topic distribution for every speech
topic_distribution = loaded_lda_model.get_document_topics(corpus, minimum_probability=0.2) # threshold
topics_list = [dict(distribution) for distribution in topic_distribution]

In [86]:

# add topic distribution as new column
df['topic_distribution'] = topics_list

In [171]:
df.head()

,text,date,speaker_name,speaker_gender,speaker_role,speaker_party,comments,id,speaker_id,speech_length,topic_distribution
0,Sehr geehrter Herr Alterspräsident! So muss ...,2021-10-26,Gabriele Katzmarek,female,None,SPD,[(Beifall bei der SPD sowie bei Abgeordneten d...,SP-2021-0,1299,2593,{68: 0.3588338}
1,Herr Präsident! Liebe Kolleginnen und Kolleg...,2021-10-26,Stefan Müller,male,None,CDU/CSU,[(Beifall bei der CDU/CSU sowie bei Abgeordnet...,SP-2021-1,4028,3796,{68: 0.46662897}
2,Sehr geehrter Herr Alterspräsident Wolfgang ...,2021-10-26,Britta Haßelmann,female,None,BÜNDNIS 90/DIE GRÜNEN,[(Beifall beim BÜNDNIS 90/DIE GRÜNEN und bei d...,SP-2021-2,451,4301,{68: 0.2632997}
3,Sehr geehrter Herr Präsident! Meine lieben K...,2021-10-26,Marco Buschmann,male,None,FDP,"[(Jan Korte [DIE LINKE]: Oder Jugendweihe!), (...",SP-2021-3,3083,4555,{}
4,Herr Alterspräsident! Lassen Sie mich zunäch...,2021-10-26,Stephan Brandner,male,None,AfD,"[(Beifall bei der AfD), (Beifall bei Abgeordne...",SP-2021-4,4055,5996,{68: 0.3100206}


In [ ]:
# save new dataframe containing topics
df.to_pickle('data_topics.pkl')